# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
# Do all imports and installs here
import pandas as pd
import pyspark.sql.functions as f
from IPython.display import display, HTML


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

In this project we will be building analytics database using [Yelp dataset](https://www.yelp.com/dataset). This analytics table can be used to answer questions like: 
- How different discount, customer relationship programs, etc on reviews. So basically data analyst should be able to run a query and see how the reviews score changes during program time vs other times
- Have ability to fitler out negative(less than average review score for given business)reviews and find top used words/phrases
- Find top users that provided most value to the business using reviews/tips

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

Dataset contains a number of of newline delimeted json files.

#### Dataset description 
From [the dataset description](https://www.yelp.com/dataset/documentation/main)

Each file is composed of a single object type, one JSON-object per-line.

Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.

Note: the follow examples contain inline comments, which are technically not valid JSON. This is done here to simplify the documentation and explaining the structure, the JSON files you download will not contain any comments and will be fully valid JSON.

Sources:

- business.json - Contains business data including location data, attributes, and categories.

```json 
{
    // string, 22 character unique string business id
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, the business's name
    "name": "Garaje",

    // string, the full address of the business
    "address": "475 3rd St",

    // string, the city
    "city": "San Francisco",

    // string, 2 character state code, if applicable
    "state": "CA",

    // string, the postal code
    "postal code": "94107",

    // float, latitude
    "latitude": 37.7817529521,

    // float, longitude
    "longitude": -122.39612197,

    // float, star rating, rounded to half-stars
    "stars": 4.5,

    // integer, number of reviews
    "review_count": 1198,

    //TODO: convert to boolean

    // integer, 0 or 1 for closed or open, respectively
    "is_open": 1,

    //TODO: do we need this?
    
    // object, business attributes to values. note: some attribute values might be objects
    "attributes": {
        "RestaurantsTakeOut": true,
        "BusinessParking": {
            "garage": false,
            "street": true,
            "validated": false,
            "lot": false,
            "valet": false
        },
    },

    // an array of strings of business categories
    "categories": [
        "Mexican",
        "Burgers",
        "Gastropubs"
    ],

    // an object of key day to value hours, hours are using a 24hr clock
    "hours": {
        "Monday": "10:00-21:00",
        "Tuesday": "10:00-21:00",
        "Friday": "10:00-21:00",
        "Wednesday": "10:00-21:00",
        "Thursday": "10:00-21:00",
        "Sunday": "11:00-18:00",
        "Saturday": "10:00-21:00"
    }
}
```
- review.json - Contains full review text data including the user_id that wrote the review and the business_id the review is written for.
```json
{
    // string, 22 character unique review id
    "review_id": "zdSx_SD6obEhz9VrW9uAWA",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // integer, star rating
    "stars": 4,

    // string, date formatted YYYY-MM-DD
    "date": "2016-03-09",

    // string, the review itself
    "text": "Great place to hang out after work: the prices are decent, and the ambience is fun. It's a bit loud, but very lively. The staff is friendly, and the food is good. They have a good selection of drinks.",

    //TODO: check min max for next values
    
    // integer, number of useful votes received
    "useful": 0,

    // integer, number of funny votes received
    "funny": 0,

    // integer, number of cool votes received
    "cool": 0
}
```
- user.json - User data including the user's friend mapping and all the metadata associated with the user.
```json
{
    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, the user's first name
    "name": "Sebastien",

    // integer, the number of reviews they've written
    "review_count": 56,

    // string, when the user joined Yelp, formatted like YYYY-MM-DD
    "yelping_since": "2011-01-01",

    // array of strings, an array of the user's friend as user_ids
    "friends": [
        "wqoXYLWmpkEH0YvTmHBsJQ",
        "KUXLLiJGrjtSsapmxmpvTA",
        "6e9rJKQC3n0RSKyHLViL-Q"
    ],

    // integer, number of useful votes sent by the user
    "useful": 21,

    // integer, number of funny votes sent by the user
    "funny": 88,

    // integer, number of cool votes sent by the user
    "cool": 15,

    // integer, number of fans the user has
    "fans": 1032,

    // array of integers, the years the user was elite
    "elite": [
        2012,
        2013
    ],

    //TODO: do we need this?
    
    // float, average rating of all reviews
    "average_stars": 4.31,

    // integer, number of hot compliments received by the user
    "compliment_hot": 339,

    // integer, number of more compliments received by the user
    "compliment_more": 668,

    // integer, number of profile compliments received by the user
    "compliment_profile": 42,

    // integer, number of cute compliments received by the user
    "compliment_cute": 62,

    // integer, number of list compliments received by the user
    "compliment_list": 37,

    // integer, number of note compliments received by the user
    "compliment_note": 356,

    // integer, number of plain compliments received by the user
    "compliment_plain": 68,

    // integer, number of cool compliments received by the user
    "compliment_cool": 91,

    // integer, number of funny compliments received by the user
    "compliment_funny": 99,

    // integer, number of writer compliments received by the user
    "compliment_writer": 95,

    // integer, number of photo compliments received by the user
    "compliment_photos": 50
}
```

- checkin.json - Checkins on a business.
```json
{
    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg"

    // string which is a comma-separated list of timestamps for each checkin, each with format YYYY-MM-DD HH:MM:SS
    "date": "2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02"
}
```
- tip.json - Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.
```json
{
    // string, text of the tip
    "text": "Secret menu - fried chicken sando is da bombbbbbb Their zapatos are good too.",

    // string, when the tip was written, formatted like YYYY-MM-DD
    "date": "2013-09-20",

    // integer, how many compliments it has
    "compliment_count": 172,

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "49JhAJh8vSQ-vM4Aourl0g"
}
```

In [4]:
# Read in the data here

In [6]:
from pyspark.sql import SparkSession

# config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
%time
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

prev = spark.conf.get("spark.sql.execution.arrow.enabled")  # Keep its default value.
# ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [7]:
uniq(df):
    for c in df.columns
        

SyntaxError: invalid syntax (3957546960.py, line 1)

In [8]:
def info(df):
    display(df.summary().toPandas())
    print("Unique values:")
    k=[c:df.select(c).distinct().count() for c in df.columns]
        
#     print("Nan values:")
#     df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()
#     print("Null values:")
#     df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in 
#            df.columns]).toPandas().T

SyntaxError: invalid syntax (3554035632.py, line 4)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


#### Businesses

In [9]:
%%time
df=spark.read.json("./data/yelp_academic_dataset_business.json")
df.cache()
info(df)
# kdf=df.toPandas()

AnalysisException: Path does not exist: file:/home/babkamen/git/udacity-big-data-engineer-nanodegree/capstone-project/data/yelp_academic_dataset_business.json

In [ ]:
kdf.shape

In [ ]:
print("Missing values")
c=kdf.isnull().sum()
print(c[c>0])

In [ ]:
df.select([col[0] for col in df.dtypes if col[1] != 'string']).describe().toPandas()

In [ ]:
kdf.describe(include="all")

In [ ]:
df['city'].duplicated().sum()

#### Reviews


In [ ]:
df=spark.read.json("./data/yelp_academic_dataset_review.json")
df.createOrReplaceTempView("reviews")

In [ ]:
# https://stackoverflow.com/questions/44413132/count-the-number-of-missing-values-in-a-dataframe-spark
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()


#### Count empty values 

In [ ]:
#https://towardsdatascience.com/data-prep-with-spark-dataframes-3629478a1041
df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()

#### Count nulls

In [ ]:
df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in 
           df.columns]).toPandas().T

In [ ]:
df.filter(f.isnan("name")).toPandas()

In [ ]:
"{:,}".format(df.count())

In [ ]:
spark.sql("SELECT * FROM reviews WHERE text='' LIMIT 10").toPandas()

In [ ]:
df.limit(5).toPandas().head()

#### Users

In [ ]:
%%time
df=spark.read.json("./data/yelp_academic_dataset_user.json")

In [ ]:
#https://stackoverflow.com/questions/3154460/python-human-readable-large-numbers

"{:,}".format(df.count())

In [ ]:
df.summary().toPandas()

In [ ]:
df.limit(5).toPandas().head()

#### Cleaning Steps
Document steps necessary to clean the data

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.